In [12]:
def getreccomendation(*symptoms):

    import pandas as pd #Analysis 
    import matplotlib.pyplot as plt #Visulization
    import seaborn as sns #Visulization
    import numpy as np #Analysis 
    from fuzzywuzzy import fuzz

    df_train = pd.read_csv("drugsComTrain_raw.csv", parse_dates=["date"])
    df_test = pd.read_csv("drugsComTest_raw.csv", parse_dates=["date"])

    df_all = pd.concat([df_train,df_test])

    df_disease = pd.read_csv("dia_t_clean.csv", encoding='latin-1')
    df_symptoms = pd.read_csv("sym_t.csv")
    df_dis_sym = pd.read_csv("diffsydiw.csv")

    df_dis_sym = df_dis_sym[['did', 'syd', 'wei']]
    df_dis_sym = df_dis_sym.sort_values(axis=0, by=['did'])
    df_dis_sym = df_dis_sym.reset_index(drop=True)
    df_dis_sym = df_dis_sym.rename(columns={"did": "disease", "syd": "symptom", "wei": "weight"})
    df_dis_sym = df_dis_sym.drop(axis=0, index=5568)

    disease_list = []
    for item in df_dis_sym.iloc[:, 0]:
        if item not in disease_list:
            disease_list.append(item)




    df_dis_sym_c = pd.DataFrame(columns={'disease', 'symptoms'})
    sym_lst = []
    for row in range(len(df_dis_sym)):
        if row == 0:
            sym_lst.append(df_dis_sym.iloc[row, 1])
        elif row == len(df_dis_sym)-1:
            sym_lst = sorted(sym_lst)
            df_dis_sym_c = df_dis_sym_c.append({'disease': df_dis_sym.iloc[row, 0], 'symptoms': sym_lst}, ignore_index=True)
        elif (df_dis_sym.iloc[row, 0] == df_dis_sym.iloc[row-1, 0]):
            sym_lst.append(df_dis_sym.iloc[row, 1])
        elif (df_dis_sym.iloc[row, 0] != df_dis_sym.iloc[row-1, 0]):
            sym_lst = sorted(sym_lst)
            df_dis_sym_c = df_dis_sym_c.append({'disease': df_dis_sym.iloc[row-1, 0], 'symptoms': sym_lst}, ignore_index=True)
            sym_lst = [df_dis_sym.iloc[row, 1]]
    columnsTitles=["disease","symptoms"]
    df_dis_sym_c=df_dis_sym_c.reindex(columns=columnsTitles)


    symp_dict = {}
    for i in range(len(df_symptoms)):
        symp_dict[df_symptoms.iloc[i, 0]] = df_symptoms.iloc[i, 1]

    symp_dict2 = {}
    for i in range(len(df_symptoms)):
        symp_dict2[df_symptoms.iloc[i, 1]] = df_symptoms.iloc[i, 0]

    dis_dict = {}
    for i in range(len(df_disease)):
        dis_dict[df_disease.iloc[i, 0]] = df_disease.iloc[i, 1]

    dis_dict2 = {}
    for i in range(len(df_disease)):
        dis_dict2[df_disease.iloc[i, 1]] = df_disease.iloc[i, 0]

    def return_disease2(*symptoms):
        top_five = []
        value_lst = []
        per_match_dic = {}
        for row in range(len(df_dis_sym_c)):
            match = 0
            total = 0
            for symptom in symptoms:
                lst = df_dis_sym_c.iloc[row, 1]
                if symp_dict2[symptom] in lst:
                    match += 1
                    total += 1
                else:
                    total += 1
            per_match = match/total
            disease_index = df_dis_sym_c.iloc[row, 0]
            disease = dis_dict[disease_index]
            per_match_dic[(disease, disease_index)] = per_match
        for value in per_match_dic.values():
            value_lst.append(value)
        value_lst = sorted(value_lst)
        value_lst_top = value_lst[-1:-6:-1]
        for (disease, index), value in per_match_dic.items():
            if value in value_lst_top:
                top_five.append(((disease, index), value))
        return top_five




    from collections import Counter 
    counter = Counter(df_all['condition'])
    conditions = counter.most_common()
    conlist = []
    reslist= []
    for i in range(0,len(conditions)):
        conlist.append(conditions[i][0])
    result = return_disease2(*symptoms)
    for i in range(0,len(result)):
        reslist.append(result[i][0][0])
    conlist = [i for i in conlist if type(i) is not float]


    #input shuens list and the list of all the conditions in the df
    def datatrsf(diseases, newconlist):
        reslist=[]
        for i in range(0,len(diseases)):
            reslist.append(diseases[i][0][0])



        val = []
        finallist = []
        finallistindex  = []
        for x in range(0,len(reslist)):
            val= []
            for y in range(0,len(newconlist)):
                val.append(fuzz.partial_ratio(reslist[x],newconlist[y]))
            values = np.array(val)
            searchval = max(val)
            ii = np.where(values == searchval)[0]
            for n in ii:
                finallist.append([newconlist[n],val[n],x])  
        return finallist


    def datatrsf1(diseases , firstout):
        display = []
        reslist=[]
        for i in range(0,len(diseases)):
            reslist.append(diseases[i][0][0])

        if(len(reslist)>5):
            for p in range(0,len(firstout)):
                    constr = firstout[p][0]
                    if(fuzz.ratio(constr, reslist[firstout[p][2]])>65):
                        display.append(firstout[p])
            if(len(display)<3):
                return firstout
        else:
            return firstout

        return display


    def outputclean(out):
        new_list = []
        for i in range(len(out)):
            if (out[i][0] != 'me'):
                new_list.append(out[i])
        return new_list



    def get_prob(result, new_list):
        for i in range(len(new_list)):
            num = new_list[i][2]
            new_list[i].append(result[num][1])
        for i in range(len(new_list)):
            del new_list[i][2]
            del new_list[i][1]
        return new_list


    def return_disease2(*symptoms):
        top_five = []
        value_lst = []
        per_match_dic = {}
        for row in range(len(df_dis_sym_c)):
            match = 0
            total = 0
            for symptom in symptoms:
                lst = df_dis_sym_c.iloc[row, 1]
                if symp_dict2[symptom] in lst:
                    match += 1
                    total += 1
                else:
                    total += 1
            per_match = match/total
            disease_index = df_dis_sym_c.iloc[row, 0]
            disease = dis_dict[disease_index]
            per_match_dic[(disease, disease_index)] = per_match
        for value in per_match_dic.values():
            value_lst.append(value)
        value_lst = sorted(value_lst)
        value_lst_top = value_lst[-1:-6:-1]
        for (disease, index), value in per_match_dic.items():
            if value in value_lst_top:
                top_five.append(((disease, index), value))
        return top_five


    def return_dis_match(*symptoms, newconlist):
        lst = return_disease2(*symptoms)
        data = datatrsf(lst, newconlist)
        data1 = outputclean(data)
        data2 = datatrsf1(lst, data1)
        data3 = get_prob(lst, data2)
        return data3

    return return_dis_match(*symptoms, newconlist=conlist)


In [11]:
getreccomendation('Bleeding tendency')

[['Aplastic Anemia', 1.0],
 ['Hemophilia A', 1.0],
 ['Hemophilia B', 1.0],
 ['Leukemia', 1.0],
 ['Myelodysplastic Syndrome', 1.0],
 ["von Willebrand's Disease", 1.0]]